# Reaction-network: Enumerators (Demo Notebook 1)

#### <u> Author:</u> Matthew McDermott (_UC Berkeley / Lawerence Berkeley National Laboratory_)
Last Updated: 08/31/23

In this notebook, we cover reaction enumeration in the _reaction-network_ package. The reaction enumerators are the core of the package and supply all reaction data to be used in reaction network construction and downstream analyses. This is the recommended starting point for working with the codebase. Please see the next notebook(s) for further applications (such as network construction).

If you use this package in your work, please consider citing the following paper:

    McDermott, M. J., Dwaraknath, S. S., and Persson, K. A. (2021). A graph-based network for predicting chemical reaction pathways in solid-state materials synthesis. Nature Communications, 12(1). https://doi.org/10.1038/s41467-021-23339-x

## 1. Imports
First, make sure the reaction-network package is installed. Run `pip install reaction-network` in your terminal.

**NOTE**: You may also need to run/update the Materials Project API. Run `pip install --upgrade mp-api`.

In [3]:
import logging
from pprint import pprint

from mp_api.client import MPRester
from pymatgen.core.periodic_table import Element

from rxn_network.core import Composition
from rxn_network.costs.functions import Softplus
from rxn_network.enumerators.basic import BasicEnumerator, BasicOpenEnumerator
from rxn_network.enumerators.minimize import MinimizeGibbsEnumerator, MinimizeGrandPotentialEnumerator
from rxn_network.entries.entry_set import GibbsEntrySet
from rxn_network.reactions.reaction_set import ReactionSet

%load_ext autoreload
%autoreload 2

## 2. Downloading and modifying entries

We will work with an example chemical system: yttrium (Y), manganese (Mn), and oxygen (o).

First, we need to acquire thermodynamic data for phases in this system from the Materials Project (MP), a computed materials database containing calculations for 150,000+ materials.

In [4]:
with MPRester() as mpr:  # insert your Materials Project API key if it's not stored as an environment variable
    entries = mpr.get_entries_in_chemsys("Y-Mn-O")

Retrieving ThermoDoc documents:   0%|          | 0/137 [00:00<?, ?it/s]

A unique feature of the `reaction-network` package is the `GibbsEntrySet` class.

This class allows us to automatically convert `ComputedStructureEntry` objects downloaded from the MP database into `GibbsComputedEntry` objects, where DFT-calculated energies have been converted to an AI-estimated equivalent values of the Gibbs free energies of formation, $\Delta G_f$ for all entries at the specified temperature.

This will also include some experimental thermochemistry data (e.g., NIST-JANAF). For more information, check out the citation in the documentation for `GibbsComputedEntry`.

In [6]:
temp = 900  # units: Kelvin
gibbs_entries = GibbsEntrySet.from_computed_entries(entries, temperature=temp, include_nist_data=True)

We can print the entries by calling `.entries` or `.entries_list`:

In [11]:
gibbs_entries.entries_list[:5]  # the first five entries in the Y-Mn-O system (in alphabetical order)

[GibbsComputedEntry | mp-723285-GGA | O8 (O2)
 Gibbs Energy (900 K) = 0.0000,
 GibbsComputedEntry | mp-1238773-GGA+U | Mn1 O1 (MnO)
 Gibbs Energy (900 K) = -2.6169,
 GibbsComputedEntry | mp-1238899-GGA+U | Mn1 O1 (MnO)
 Gibbs Energy (900 K) = -2.9922,
 GibbsComputedEntry | mp-25223-GGA+U | Mn1 O2 (MnO2)
 Gibbs Energy (900 K) = -3.7579,
 GibbsComputedEntry | mp-796077-GGA+U | Mn1 O2 (MnO2)
 Gibbs Energy (900 K) = 0.6017]

The `GibbsEntrySet` class has many helpful functions, such as the following `filter_by_stability()` function, which automatically removes entries which are a specified energy per atom above the convex hull of stability:

In [13]:
filtered_entries = gibbs_entries.filter_by_stability(0.025)

You should now see a much shorter list of entries within the Y-Mn-O system (< 25 meV/atom below the hull):

In [15]:
print(f"{len(gibbs_entries)} (unfiltered) -> {len(filtered_entries)} (filtered)")

108 (unfiltered) -> 13 (filtered)


Another useful function is the `get_min_entry_by_formula` function. This automatically finds the entry with the lowest energy matching the provided formula (composition). This is useful for querying the ground-state polymorph when there are many entries for a particular composition.

In [18]:
gibbs_entries.get_min_entry_by_formula("Mn2O3")

GibbsComputedEntry | mp-1172875-GGA+U | Mn32 O48 (Mn2O3)
Gibbs Energy (900 K) = -121.1202

## 3. Running enumerators

Now that we've discussed creating a set of entries, we will learn how to enumerate reactions from those entries. 

There are four distinct enumerator classes contained within `rxn_network.enumerators`: These are:

**(basic)**
1. `BasicEnumerator`: uses a _combinatorial_ approach to identify all possible (closed) reactions within a set of entries.
2. `BasicOpenEnumerator`: uses a _combinatorial_ approach to identify all **open** reactions within a set of entries and a list of specified open entries/elements.
   
**(minimize)**

3. `MinimizeGibbsEnumerator`: uses a _thermodynamic_ approach to identify all reactions within a set of entries that are predicted by minimizing Gibbs free energy between a set of two reacting phases touching at an interface.
4. `MinimizeGrandPotentialEnumerator`: uses a _thermodynamic_ approach to identify all reactions within a set of entries that are predicted by minimizing the grand potential energy between a set of two reacting phases touching at an interface with an **open** element at a specified chemical potential.

There is no "correct" enumerator to use; while the thermodynamic (i.e., minimize) enumerators may seem more logical, our thermodynamic data is not always exaclty correct -- this means that some reactions will not be enumerated.

### a) Basic enumerators
We first create a basic enumerator object by initializing one from the `BasicEnumerator` class. Let's initialize it with only default arguments:

In [55]:
be = BasicEnumerator()

The `BasicEnumerator` class, as is true for all other enumerator classes, has many arguments that will customize its output. To view helpful documentation for these arguments, press Shift+Tab with your cursor inside the parentheses in the cell above.

A key argument to the basic enumerators is the maximum reactant/product cardinality, $n$. The default is $n=2$. This setting suffices for capturing many of the reactions that make up a solid-state reaction pathway according to the pairwise interface reaction hypothesis (most powder reactions proceed through reactions of interfacial pairs). Note that it is possible to set $n=3$, however this dramatically increases the combinatorial complexity and can take a long time for more complex systems.

In general, the default arguments are good for generating a list of simple (unconstrained) reactions, as we might build for a reaction network. Run the following cell:

In [56]:
all_rxns = be.enumerate(filtered_entries)

Enumerating reactions (BasicEnumerator): 100%|█| 1/1 [00:00<00:00,  1.96it/s]


You may notice that something called `Ray` is initialized. This is a python library for parallelizing functions and is used by the `reaction-network` code to parallelize enumeration. 

The cell above should have completed somewhat quickly -- ideally within a second or two. As a result, a list of 692 generated reactions will be stored within the `all_rxns` object. Note: this number may change in the future if the MP database changes...

In [57]:
print(len(all_rxns))

692


Every time `enumerate()` is called, a `ReactionSet` will be returned. This is a memory-efficient object that can be used to store large sets of reactions.

The `ReactionSet` class stores sets of reactions as arrays. Note that the actual reaction objects can only be accessed by **iterating** through the reaction set. Lets print the "first" 10 reactions. These may be different on your side; the reactions are generated in no particualr order.

In [58]:
for count, r in enumerate(all_rxns):
    print(r)
    if count>10:
        break

2 Mn3O4 -> O2 + 6 MnO
O2 + 6 MnO -> 2 Mn3O4
0.5 Mn3O4 -> O2 + 1.5 Mn
O2 + 1.5 Mn -> 0.5 Mn3O4
3 Mn3O4 -> Mn5O8 + 4 MnO
Mn5O8 + 4 MnO -> 3 Mn3O4
Mn3O4 -> MnO2 + 2 MnO
MnO2 + 2 MnO -> Mn3O4
Mn3O4 -> MnO + Mn2O3
MnO + Mn2O3 -> Mn3O4
2 Mn3O4 -> Mn5O8 + Mn
Mn5O8 + Mn -> 2 Mn3O4


Looking at the list of reactions above, we see that all reactions are stoichiometrically balanced. If we look at any particular reaction object, we find that the reaction energy and uncertainty are properties which can be easily accessed:

In [59]:
r = list(all_rxns)[0]
print(r)
print(f"{round(r.energy_per_atom, 4)} ± {round(r.energy_uncertainty_per_atom, 2)} eV/atom")

2 Mn3O4 -> O2 + 6 MnO
0.2332 ± 0.07 eV/atom


What if you want to enumerate all reactions from a known set of precursors? An inefficient way to do so might be to filter the previous reaction set by your phases of interest...

However, a more efficient solution has been provided. We can supply our precursor formulas when we initialize the `BasicEnumerator` object. This will reduce the number of calculations required significantly. Lets say we have Y2O3 as a precursor:

In [60]:
be_precursors = BasicEnumerator(precursors=["Y2O3"])
y2o3_rxns_exclusive = be_precursors.enumerate(filtered_entries)

Building chunks...: 100%|████████████████████| 1/1 [00:00<00:00, 1449.31it/s]
Enumerating reactions (BasicEnumerator): 100%|█| 1/1 [00:00<00:00, 421.96it/s


In [61]:
for r in y2o3_rxns_exclusive:
    print(r)

0.6667 Y2O3 -> O2 + 1.333 Y


Note that by default, this only produces reactions which **exclusively** have the provided precursor(s). 

To include reactions that contain this precursor (and possibly others) set the `exclusive_precursors=False`:

In [62]:
be_precursors = BasicEnumerator(precursors=["Y2O3"], exclusive_precursors=False)
y2o3_rxns = be_precursors.enumerate(filtered_entries)

Enumerating reactions (BasicEnumerator): 100%|█| 1/1 [00:00<00:00,  5.05it/s]


We now get a much larger list of reactions, all of which contain Y2O3 as a precursor (and often another phase such as a manganese oxide):

In [63]:
pprint(list(y2o3_rxns)[:10])  # a sample of just 10 reactions

[0.6667 Y2O3 -> O2 + 1.333 Y,
 Y2O3 + Mn2O3 -> 2 YMnO3,
 MnO2 + 0.5 Y2O3 -> 0.5 Y2Mn2O7,
 4 Mn3O4 + 0.5 Y2O3 -> YMn12 + 8.75 O2,
 3 Mn3O4 + 5 Y2O3 -> Y + 9 YMnO3,
 0.6667 Mn3O4 + 0.7778 Y2O3 -> YMn2O5 + 0.5556 Y,
 0.75 Mn3O4 + 1.625 Y2O3 -> Y + 1.125 Y2Mn2O7,
 1.667 Mn3O4 + 0.4444 Y2O3 -> Mn5O8 + 0.8889 Y,
 0.75 Mn3O4 + 0.5 Y2O3 -> Y + 2.25 MnO2,
 3 Mn3O4 + 0.5 Y2O3 -> Y + 4.5 Mn2O3]


This same approach can be used for the target phase(s) as well.

With `exclusive_targets=True`:

In [64]:
be_target = BasicEnumerator(targets=["YMnO3"], exclusive_targets=True)
ymno3_rxns_exclusive = be_target.enumerate(filtered_entries)

Enumerating reactions (BasicEnumerator): 100%|█| 1/1 [00:00<00:00, 27.16it/s]


In [65]:
for r in ymno3_rxns_exclusive:
    print(r)

Y2O3 + Mn2O3 -> 2 YMnO3


Due to the fact that the right side of the reaction can _only_ contain YMnO3, we get just **one** reaction above! 

Now, with `exclusive_targets=False` (the default):

In [66]:
be_target = BasicEnumerator(targets=["YMnO3"])
ymno3_rxns = be_target.enumerate(filtered_entries)

Enumerating reactions (BasicEnumerator): 100%|█| 1/1 [00:00<00:00,  3.65it/s]


In [67]:
pprint(list(ymno3_rxns)[:10])

[Y2O3 + Mn2O3 -> 2 YMnO3,
 YMn2O5 -> MnO2 + YMnO3,
 2 Y2Mn2O7 -> O2 + 4 YMnO3,
 3 Mn3O4 + 5 Y2O3 -> Y + 9 YMnO3,
 Mn3O4 + Y2O3 -> MnO + 2 YMnO3,
 3 Mn3O4 + 4 Y2O3 -> Mn + 8 YMnO3,
 39 Mn3O4 + 53 Y2O3 -> YMn12 + 105 YMnO3,
 0.4 Mn3O4 + 0.2 Y -> MnO + 0.2 YMnO3,
 0.6 Mn3O4 + 0.8 Y -> Mn + 0.8 YMnO3,
 7.2 Mn3O4 + 10.6 Y -> YMn12 + 9.6 YMnO3]


And finally, with multiple targets specified (e.g., YMnO3 and O2):

In [68]:
be_targets = BasicEnumerator(targets=["YMnO3", "O2"], exclusive_targets=True)
ymno3_rxns_o2 = be_targets.enumerate(filtered_entries)

Enumerating reactions (BasicEnumerator): 100%|█| 1/1 [00:00<00:00, 14.08it/s]


In [69]:
for r in ymno3_rxns_o2:
    print(r)

Y2O3 + Mn2O3 -> 2 YMnO3
2 Y2Mn2O7 -> O2 + 4 YMnO3
YMn2O5 + 0.5 Y2O3 -> 0.25 O2 + 2 YMnO3
Mn5O8 + 2.5 Y2O3 -> 0.25 O2 + 5 YMnO3
MnO2 + 0.5 Y2O3 -> 0.25 O2 + YMnO3


#### What about open gases, liquids, etc.??

In the previous cell, we showed that it was possible to specify YMnO$_3$ as a target, along with O$_2$. However, because O$_2$ is a gas, it is often desirable to include it as an **open entry** in addition to the 2 possible precursors or targets. For example, we may want to specify a reaction as follows:

$$ A + B ~ +~\textrm{O}_2 \rightarrow C + D $$
$$ \textrm{or} $$
$$ A + B \rightarrow C + D ~ +~\textrm{O}_2 $$

To do this, we use the `BasicOpenEnumerator` class, which is an extension to the original basic enumerator. All of the lessons learned above also apply to this class, although now a list of open entry formulas (e.g., `["O2"]`) must be specified:

In [71]:
be_target_open = BasicOpenEnumerator(open_phases=["O2"], targets=["YMnO3"])
ymno3_rxns_open = be_target_open.enumerate(filtered_entries)

Building chunks...: 100%|████████████████████| 3/3 [00:00<00:00, 1463.47it/s]
Enumerating reactions (BasicOpenEnumerator): 100%|█| 2/2 [00:00<00:00,  5.04i


The `BasicOpenEnumerator` generally requires more time to run as it considers a larger combinatorial space.

You will now see that the reactions from before are included in this new list, along with many other open-O$_2$ reactions -- some which even have a total of 3 reactants or 3 products. In other words, the open entry/entries do not count towards the specified maximum cardinality, $n$.

In [75]:
for r in list(ymno3_rxns_open)[:10]:  # first 10 reactions
    print(r)

YMn12 + 10.3 O2 -> 2.2 Mn5O8 + YMnO3
YMn12 + 12.5 O2 -> YMnO3 + 11 MnO2
YMn12 + 9.75 O2 -> YMnO3 + 5.5 Mn2O3
O2 + 4 Mn3O4 + 6 Y2O3 -> 12 YMnO3
O2 + 0.4 Mn3O4 + 1.2 Y -> 1.2 YMnO3
O2 + 2 Y2O3 + 4 MnO -> 4 YMnO3
O2 + 1.333 Mn + 0.6667 Y2O3 -> 1.333 YMnO3
YMn12 + 9.75 O2 + 5.5 Y2O3 -> 12 YMnO3
O2 + Y + MnO -> YMnO3
O2 + 0.6667 Mn + 0.6667 Y -> 0.6667 YMnO3


It is even possible to specify multiple open entries, allowing for more complex reactions. For example, specifiyng Y2O3 and O2 as both being open:

In [81]:
be_target_open2 = BasicOpenEnumerator(open_phases=["Y2O3", "O2"],targets=["YMnO3"])
ymno3_rxns_open2 = be_target_open2.enumerate(filtered_entries)

Building chunks...: 100%|█████████████████████| 2/2 [00:00<00:00, 668.41it/s]
Enumerating reactions (BasicOpenEnumerator): 100%|█| 6/6 [00:00<00:00,  8.39i


In [83]:
for r in list(ymno3_rxns_open2)[:10]:  # first 10 reactions
    print(r)

O2 + 4 Mn3O4 + 6 Y2O3 -> 12 YMnO3
MnO + 0.5 Y2O3 + 0.25 O2 -> YMnO3
O2 + 1.333 Mn + 0.6667 Y2O3 -> 1.333 YMnO3
O2 + 0.1026 YMn12 + 0.5641 Y2O3 -> 1.231 YMnO3
Y + 0.8333 O2 + 0.3333 Mn3O4 -> YMnO3
Y2O3 + 0.5 Mn3O4 + 0.5 YMn2O5 -> 2.5 YMnO3
Y2O3 + 0.3333 Y2Mn2O7 + 0.6667 Mn3O4 -> 2.667 YMnO3
Y2O3 + 0.25 Mn5O8 + 0.25 Mn3O4 -> 2 YMnO3
Y2O3 + 0.5 Mn3O4 + 0.5 MnO2 -> 2 YMnO3
MnO + O2 + Y -> YMnO3


This may be useful for modeling systems with 2 or more gaseous, liquid, or molten phases, such as $O_2$ and a molten salt (e.g. $LiCl$).

If the system is open to a particular element such as $O_2$, then it is generally a goood idea to model the thermodynamics of the system with the grand potential energy, $\Phi$. This can be easily done by providing the open element and chemical potential to the `ReactionSet` class. In this case, we will reinitialize the reactions:

In [93]:
grand_potential_rxns = ymno3_rxns_open2.set_chempot(open_el=Element("O"), chempot=0.0)
pprint(list(grand_potential_rxns)[:10])

[O2 + 4 Mn3O4 + 6 Y2O3 -> 12 YMnO3 (mu_O=0.0),
 MnO + 0.5 Y2O3 + 0.25 O2 -> YMnO3 (mu_O=0.0),
 O2 + 1.333 Mn + 0.6667 Y2O3 -> 1.333 YMnO3 (mu_O=0.0),
 O2 + 0.1026 YMn12 + 0.5641 Y2O3 -> 1.231 YMnO3 (mu_O=0.0),
 Y + 0.8333 O2 + 0.3333 Mn3O4 -> YMnO3 (mu_O=0.0),
 Y2O3 + 0.5 Mn3O4 + 0.5 YMn2O5 -> 2.5 YMnO3 (mu_O=0.0),
 Y2O3 + 0.3333 Y2Mn2O7 + 0.6667 Mn3O4 -> 2.667 YMnO3 (mu_O=0.0),
 Y2O3 + 0.25 Mn5O8 + 0.25 Mn3O4 -> 2 YMnO3 (mu_O=0.0),
 Y2O3 + 0.5 Mn3O4 + 0.5 MnO2 -> 2 YMnO3 (mu_O=0.0),
 MnO + O2 + Y -> YMnO3 (mu_O=0.0)]


Each of these reactions is a new class: `OpenComputedReaction` and its energy (per atom) is calculated assumning oxygen is an open element with the defined chemical potential. We will discuss this further later in the notebook.

### b) Minimize enumerators

The "minimize" enumerators produce reactions via a thermodynamic free energy minimization approach, rather than a purely combinatorial one. This means that reactions are produced directly from the compositional phase diagram, where a new convex hull is drawn connecting two compositions within a closed (Gibbs) or open (Grand Potential) system. See the `InterfacialReactivity` class within the _pymatgen_ package for more information.

**It is important to note that reactions produced with the minimize enumerators may overlap some with the basic enumerators, but the minimize enumerators have the restriction that all reactions they originally produce must have a negative reaction energy and result in a set of product phases which are stable with respect to each other (i.e. they share a facet of the phase diagram).** That being said, these enumerators (when unrestricted) will also supply the reverse (i.e. positive energy) reaction as well.

In general, this means that the basic enumerators and minimize enumerators do not perfectly overlap in their outputs, and may be used in tandem. This is recommended! Let's initialize the `MinimizeGibbsEnumerator` with all default arguments (press Shift+Tab to view documentation).

In [94]:
mge = MinimizeGibbsEnumerator()

The default arguments, as before, help produce all reactions in a set of entries, given minimal constraints.

In [95]:
rxns = mge.enumerate(filtered_entries)

Building phase diagrams (MinimizeGibbsEnumerator): 100%|█| 4/4 [00:00<00:00, 
Building chunks...: 100%|█████████████████████| 4/4 [00:00<00:00, 994.56it/s]
Enumerating reactions (MinimizeGibbsEnumerator): 100%|█| 1/1 [00:00<00:00,  4


In [97]:
for r in list(rxns)[0:10]:  # first 10 reactions
    print(r)

0.5 Mn5O8 -> Mn2O3 + 0.5 MnO2
0.5 Mn5O8 -> Mn2O3 + 0.5 MnO2
0.6667 Mn3O4 + 0.1667 O2 -> Mn2O3
0.3333 Mn3O4 + 0.3333 O2 -> MnO2
0.25 Mn + 0.25 Mn3O4 -> MnO
0.25 Mn5O8 + 0.25 Mn3O4 -> Mn2O3
0.5 Mn5O8 -> Mn2O3 + 0.5 MnO2
0.5 MnO2 + 0.5 Mn3O4 -> Mn2O3
MnO + 0.5 O2 -> MnO2
2 MnO + 0.5 O2 -> Mn2O3


And as before, we can specify various combinations of precursors and targets, as well as whether or not they should be "exclusive". Here we specify Y2O3 as a required, but non-exclusive precursor):

In [98]:
mge_precursors = MinimizeGibbsEnumerator(precursors=["Y2O3"], exclusive_precursors=False)
rxns = mge_precursors.enumerate(filtered_entries)

Building phase diagrams (MinimizeGibbsEnumerator): 100%|█| 4/4 [00:00<00:00, 
Building chunks...: 100%|█████████████████████| 4/4 [00:00<00:00, 956.73it/s]
Enumerating reactions (MinimizeGibbsEnumerator): 100%|█| 1/1 [00:00<00:00, 32


In [99]:
for r in rxns:
    print(r)

0.5 Mn3O4 + 0.5 Y2O3 -> YMnO3 + 0.5 MnO
0.3333 Mn5O8 + 0.6667 Y2O3 -> YMnO3 + 0.3333 YMn2O5
0.625 Mn5O8 + 0.5 Y2O3 -> YMn2O5 + 0.375 Mn3O4
0.6667 Mn5O8 + 0.3333 Y2O3 -> Mn2O3 + 0.6667 YMn2O5
2 MnO2 + 0.5 Y2O3 -> YMn2O5 + 0.25 O2
2.5 Mn2O3 + 0.5 Y2O3 -> YMn2O5 + Mn3O4
2 MnO2 + Y2O3 -> Y2Mn2O7
0.5 Mn2O3 + 0.5 Y2O3 -> YMnO3


And here we specify both Y2O3 and Mn2O3 as the exclusive precursors of the reaction:

In [100]:
mge_precursors = MinimizeGibbsEnumerator(precursors=["Y2O3", "Mn2O3"])
rxns = mge_precursors.enumerate(filtered_entries)

Building phase diagrams (MinimizeGibbsEnumerator): 100%|█| 4/4 [00:00<00:00, 
Building chunks...: 100%|████████████████████| 4/4 [00:00<00:00, 1056.23it/s]
Enumerating reactions (MinimizeGibbsEnumerator): 100%|█| 1/1 [00:00<00:00, 86


In [101]:
for r in rxns:
    print(r)

2.5 Mn2O3 + 0.5 Y2O3 -> YMn2O5 + Mn3O4
0.5 Mn2O3 + 0.5 Y2O3 -> YMnO3


Now we specify YMnO3 as the exclusive target of the reaction.

In [102]:
mge_targets = MinimizeGibbsEnumerator(targets=["YMnO3"], exclusive_targets=True)
rxns = mge_targets.enumerate(filtered_entries)

Building phase diagrams (MinimizeGibbsEnumerator): 100%|█| 4/4 [00:00<00:00, 
Building chunks...: 100%|████████████████████| 4/4 [00:00<00:00, 1073.54it/s]
Enumerating reactions (MinimizeGibbsEnumerator): 100%|█| 1/1 [00:00<00:00,  4


This only identifies one reaction!

In [103]:
for r in rxns:
    print(r)

0.5 Mn2O3 + 0.5 Y2O3 -> YMnO3


Lastly, we identify YMnO3 as a required target, with any number of byproducts (i.e., non-exclusive target):

In [117]:
mge_targets = MinimizeGibbsEnumerator(targets=["YMnO3"], exclusive_targets=False)
rxns = mge_targets.enumerate(filtered_entries)

Building phase diagrams (MinimizeGibbsEnumerator): 100%|█| 4/4 [00:00<00:00, 
Building chunks...: 100%|████████████████████| 4/4 [00:00<00:00, 1006.55it/s]
Enumerating reactions (MinimizeGibbsEnumerator): 100%|█| 1/1 [00:00<00:00,  4


In [118]:
for r in list(rxns)[:40]:  # first 10 reactions
    print(r, r.energy_per_atom)

0.5 Mn3O4 + 0.5 Y2O3 -> YMnO3 + 0.5 MnO -0.029691573427302026
2 Mn3O4 + Y -> YMnO3 + 5 MnO -0.42878254591180437
14 Mn3O4 + YMn12 -> YMnO3 + 53 MnO -0.2192440094858455
Mn3O4 + 2 Y2Mn2O7 -> YMnO3 + 3 YMn2O5 -0.039591749392732606
YMn12 + 7 O2 -> YMnO3 + 11 MnO -1.7478231276583749
YMn12 + 8.833 O2 -> YMnO3 + 3.667 Mn3O4 -1.7340356833050645
0.3333 Mn5O8 + 0.6667 Y2O3 -> YMnO3 + 0.3333 YMn2O5 -0.08969957041268571
0.2857 Y + 0.7143 YMn2O5 -> YMnO3 + 0.1429 Mn3O4 -0.3571228340679312
0.3333 Y + 0.6667 YMn2O5 -> YMnO3 + 0.3333 MnO -0.4285888970633333
0.3333 Y + 0.5 Y2Mn2O7 -> YMnO3 + 0.1667 Y2O3 -0.4614723709416988
0.6667 Mn5O8 + Y -> YMnO3 + 2.333 MnO -0.7879473221346187
1.25 Mn5O8 + Y -> YMnO3 + 1.75 Mn3O4 -0.501668075281015
2.5 MnO2 + Y -> YMnO3 + 0.5 Mn3O4 -0.9956040509383094
2 MnO2 + Y -> YMnO3 + MnO -1.1509221691594167
5 Mn2O3 + Y -> YMnO3 + 3 Mn3O4 -0.31058993968934345
2 Mn2O3 + Y -> YMnO3 + 3 MnO -0.6444707532704925
2 MnO + YMn2O5 -> YMnO3 + Mn3O4 -0.0355947108916491
1.5 MnO + 0.5 Y2Mn2O

Notice that there are many more complex reactions here -- all but the previous one we identified feature a byproduct phase that is required to balance the reaction.

#### Open entries

Once again, as before, we can do all the same analysis with open entries. As previously mentioned, the grand potential is used as the thermodynamic free energy which is minimized:

$$ \Phi = G - \mu_iN_i $$

Where $i$ is the open species with chemical potential $\mu_i$. The amount of the open species in a particular phase, $N_i$, is used to adjust the energy of that phase.

Let's create a `MinimizeGrandPotentialEnumerator` with default arguments and oxygen as the open element with chemical potential $\mu=0$. Note that this is equivalent to $\mu=\mu_O^0$ due to the fact that we are working with Gibbs free energies of formation:

In [119]:
mgpe = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=0)
open_rxns = mgpe.enumerate(filtered_entries)

Building phase diagrams (MinimizeGrandPotentialEnumerator): 100%|█| 3/3 [00:0
Building chunks...: 100%|█████████████████████| 3/3 [00:00<00:00, 774.00it/s]
Enumerating reactions (MinimizeGrandPotentialEnumerator): 100%|█| 1/1 [00:00<


In [123]:
for r in list(open_rxns)[:10]:  # first 10 reactions
    print(r)

0.3333 Mn3O4 + 0.3333 O2 -> MnO2
0.3333 Mn3O4 + 0.3333 O2 -> MnO2
MnO + 0.5 O2 -> MnO2
MnO + 0.5 O2 -> MnO2
Mn + O2 -> MnO2
Mn + O2 -> MnO2
0.2 Mn5O8 + 0.2 O2 -> MnO2
0.2 Mn5O8 + 0.2 O2 -> MnO2
0.5 Mn2O3 + 0.25 O2 -> MnO2
0.5 Mn2O3 + 0.25 O2 -> MnO2


Note that this enumerator often returns duplicates particularly with terminal reactions.

To ensure that duplicates are not returned, we can initialize the enumerator with a different flag:

In [124]:
mgpe = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=0, filter_duplicates=True)
open_rxns = mgpe.enumerate(filtered_entries)

for r in list(open_rxns)[:10]:  # first 10 reactions
    print(r)

Building phase diagrams (MinimizeGrandPotentialEnumerator): 100%|█| 3/3 [00:0
Building chunks...: 100%|█████████████████████| 3/3 [00:00<00:00, 771.01it/s]
Enumerating reactions (MinimizeGrandPotentialEnumerator): 100%|█| 1/1 [00:00<
Filtering duplicates: 100%|█████████████████| 8/8 [00:00<00:00, 10869.59it/s]

MnO + 0.5 O2 -> MnO2
Mn + O2 -> MnO2
0.5 Mn2O3 + 0.25 O2 -> MnO2
0.3333 Mn3O4 + 0.3333 O2 -> MnO2
2 Y + 1.5 O2 -> Y2O3
0.2 Mn5O8 + 0.2 O2 -> MnO2
2 YMnO3 + 0.5 O2 -> Y2Mn2O7
2 MnO2 + 2 Y + 1.5 O2 -> Y2Mn2O7
2 MnO2 + Y + 0.5 O2 -> YMn2O5
4 MnO2 + 2 Y2Mn2O7 -> O2 + 4 YMn2O5


Also note that the reaction objects returned are **NOT** (by default) configured to report their energy as a change in the grand potential. To configure this, we need to transform these reactions to `OpenComputedReaction` objects. This class allows for easy specification of reactions where one of the elements is assigned a chemical potential.

We did this earlier through a convenience constructor method:

In [126]:
new_open_rxns = open_rxns.set_chempot(Element("O"), chempot=0.0)

We should now see that `(mu_O=0.0)` appears in the repr of the object:

In [127]:
sample_open_rxn = list(new_open_rxns)[0]
print(sample_open_rxn.__class__.__name__)

sample_open_rxn

OpenComputedReaction


MnO + 0.5 O2 -> MnO2 (mu_O=0.0)

We can also, as before, customize which precursors and targets are specified:

In [132]:
mgpe_precursors = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=0,
                                                   precursors=["Y2O3"], exclusive_precursors=False, filter_duplicates=True)
open_rxns_precursors = mgpe_precursors.enumerate(filtered_entries)

Building phase diagrams (MinimizeGrandPotentialEnumerator): 100%|█| 3/3 [00:0
Building chunks...: 100%|█████████████████████| 3/3 [00:00<00:00, 771.25it/s]
Enumerating reactions (MinimizeGrandPotentialEnumerator): 100%|█| 1/1 [00:00<
Filtering duplicates: 0it [00:00, ?it/s]


In [133]:
for r in open_rxns_precursors:
    print(r)

8 MnO2 + 2 Y2O3 -> O2 + 4 YMn2O5
2 MnO + Y2O3 + O2 -> Y2Mn2O7
2 MnO + 0.5 Y2O3 + 0.75 O2 -> YMn2O5
0.4 Mn5O8 + Y2O3 + 0.4 O2 -> Y2Mn2O7
0.4 Mn5O8 + 0.5 Y2O3 + 0.15 O2 -> YMn2O5
0.6667 Mn3O4 + Y2O3 + 0.6667 O2 -> Y2Mn2O7
0.6667 Mn3O4 + 0.5 Y2O3 + 0.4167 O2 -> YMn2O5
2 Mn + Y2O3 + 2 O2 -> Y2Mn2O7
2 Mn + 0.5 Y2O3 + 1.75 O2 -> YMn2O5
Mn2O3 + Y2O3 + 0.5 O2 -> Y2Mn2O7
Mn2O3 + 0.5 Y2O3 + 0.25 O2 -> YMn2O5
0.1667 YMn12 + 0.9167 Y2O3 + 2.125 O2 -> Y2Mn2O7
0.1667 YMn12 + 0.4167 Y2O3 + 1.875 O2 -> YMn2O5
0.5 Y2O3 + YMn2O5 + 0.25 O2 -> Y2Mn2O7


In [134]:
mgpe_precursors = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=0,
                                                   targets=["Y2Mn2O7"], filter_duplicates=True)
open_rxns_targets = mgpe_precursors.enumerate(filtered_entries)

Building phase diagrams (MinimizeGrandPotentialEnumerator): 100%|█| 3/3 [00:0
Building chunks...: 100%|█████████████████████| 3/3 [00:00<00:00, 842.46it/s]
Enumerating reactions (MinimizeGrandPotentialEnumerator): 100%|█| 1/1 [00:00<
Filtering duplicates: 100%|██████████████████| 1/1 [00:00<00:00, 1720.39it/s]


In [135]:
for r in open_rxns_targets:
    print(r)

2 YMnO3 + 0.5 O2 -> Y2Mn2O7
2 MnO2 + 2 Y + 1.5 O2 -> Y2Mn2O7
2 MnO + 2 Y + 2.5 O2 -> Y2Mn2O7
2 MnO + Y2O3 + O2 -> Y2Mn2O7
0.4 Mn5O8 + 2 Y + 1.9 O2 -> Y2Mn2O7
0.4 Mn5O8 + Y2O3 + 0.4 O2 -> Y2Mn2O7
0.6667 Mn3O4 + 2 Y + 2.167 O2 -> Y2Mn2O7
0.6667 Mn3O4 + Y2O3 + 0.6667 O2 -> Y2Mn2O7
2 Mn + 2 Y + 3.5 O2 -> Y2Mn2O7
2 Mn + Y2O3 + 2 O2 -> Y2Mn2O7
Mn2O3 + 2 Y + 2 O2 -> Y2Mn2O7
Mn2O3 + Y2O3 + 0.5 O2 -> Y2Mn2O7
0.1667 YMn12 + 1.833 Y + 3.5 O2 -> Y2Mn2O7
0.1667 YMn12 + 0.9167 Y2O3 + 2.125 O2 -> Y2Mn2O7
Y + YMn2O5 + O2 -> Y2Mn2O7
0.5 Y2O3 + YMn2O5 + 0.25 O2 -> Y2Mn2O7


Note that setting the chemical potential to a value outside of the range of stability of the target (e.g., `mu_O = -3`, causes the enumerator to yield no reactions:

In [136]:
mgpe_precursors = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=-3,
                                                   targets=["Y2Mn2O7"])
open_rxns_targets = mgpe_precursors.enumerate(filtered_entries)

Building phase diagrams (MinimizeGrandPotentialEnumerator): 100%|█| 3/3 [00:0
Building chunks...: 100%|█████████████████████| 3/3 [00:00<00:00, 805.15it/s]
Enumerating reactions (MinimizeGrandPotentialEnumerator): 100%|█| 1/1 [00:00<


In [137]:
print(len(open_rxns_targets))

0


This concludes the introduction of the four enumerator classes. In general, the configurations we showed are all that you will really need to use the `reaction-network` package. You are free to explore the other enumerator arguments; note that many of these often do not need to be changed.

It is encouraged for you to try out what you have learned above on your own system of interest!

## 4. Running enumerators with the _jobflow_ package

Running reaction enumeration calculations in high-throughput can be challenging. Often, we want to be able to enumerate reactions as part of some workflow and keep track of them using a database structure.

For this reason, we turn to the _jobflow_ package, which is a fantastic toolit for developing and running computational workflows. The _reaction-network_ package has several jobs (and flows) written using jobflow that can be either run directly or strung together as part of custom workflows.

Let's run one of the jobs via `ReactionEnumerationMaker` locally on your computer. This job does exactly what we were doing above -- run a list of enumerators on a provided entry set and collect the reactions in a `ReactionSet`. First, we import the job maker and run function:

In [167]:
from jobflow.managers.local import run_locally
from rxn_network.jobs.core import ReactionEnumerationMaker

Now, all we need to do is initialize the `Maker` and use the `make()` function with a list of enumerators and our entry set. The enumerators can be customized when they are created and this will be passed to the jbo:

In [166]:
be = BasicEnumerator()  # put your desired settings here
boe = BasicOpenEnumerator(open_phases=["O2"])

maker = ReactionEnumerationMaker()
job = maker.make([be, boe], filtered_entries)

The job can now be run either locally (as shown here) or launched on a remote workstation using the _fireworks_ package. 

Ti configure launching these jobs on other machines, or how the data is stored in various databases, then please see the `jobflow` documentation here: https://materialsproject.github.io/jobflow/.

**WARNING:** You must have jobflow properly configured in order to run this successfully (see link above). This means configuring your `jobflow.yaml` file and having a working connection to the database specified. For this particular job, you should also specify an additional data store called _"rxns"_ in your jobflow.yaml file. I typically make this a `GridFSStore` as the outputs can get quite large.

In [168]:
output = run_locally(job)

2023-08-31 11:05:58,807 INFO Started executing jobs locally
2023-08-31 11:05:58,809 INFO Starting job - ReactionEnumerationMaker.make (f78cec40-8e7d-415c-84c6-d6e5db037354)
2023-08-31 11:05:58,811 INFO rxn_network.jobs.core Running enumerators...
2023-08-31 11:05:58,811 INFO rxn_network.enumerators.utils Running BasicEnumerator


Enumerating reactions (BasicEnumerator): 100%|█| 1/1 [00:00<00:00,  1.91it/s]

2023-08-31 11:05:59,348 INFO rxn_network.enumerators.utils Adding 692 reactions to reaction set
2023-08-31 11:05:59,348 INFO rxn_network.enumerators.utils Running BasicOpenEnumerator



Building chunks...: 100%|████████████████████| 3/3 [00:00<00:00, 1450.65it/s]
Enumerating reactions (BasicOpenEnumerator): 100%|█| 2/2 [00:00<00:00,  2.37i

2023-08-31 11:06:00,204 INFO rxn_network.enumerators.utils Adding 204 reactions to reaction set
2023-08-31 11:06:00,205 INFO rxn_network.enumerators.utils Completed reaction enumeration. Filtering duplicates...



Filtering duplicates: 100%|████████████████| 22/22 [00:00<00:00, 3802.16it/s]

2023-08-31 11:06:00,221 INFO rxn_network.enumerators.utils Completed duplicate filtering.
2023-08-31 11:06:00,221 INFO rxn_network.jobs.core Building task document...


2023-08-31 11:06:04,106 INFO Finished job - ReactionEnumerationMaker.make (f78cec40-8e7d-415c-84c6-d6e5db037354)
2023-08-31 11:06:04,110 INFO Finished executing jobs locally


When the job completes, it will store its data in the `JobStore` database. However, the output reactions can also be accessed locally here in the notebook. To do this, access the `rxns` property of the output `EnumeratorTaskDocument`:

In [179]:
response = output[job.uuid][1].output
response.dict()

{'task_label': 'enumerate_reactions',
 'last_updated': '2023-08-31 18:06:00.222354',
 'rxns': <rxn_network.reactions.reaction_set.ReactionSet at 0x357b99630>,
 'targets': [],
 'elements': [Element Y, Element Mn, Element O],
 'chemsys': 'Y-Mn-O',
 'added_elements': None,
 'added_chemsys': None,
 'enumerators': [<rxn_network.enumerators.basic.BasicEnumerator at 0x357da0d00>,
  <rxn_network.enumerators.basic.BasicOpenEnumerator at 0x357da3f40>]}

In [181]:
for r in list(response.rxns)[0:10]:
    print(r)

O2 + 4 YMnO3 -> 2 Y2Mn2O7
2 Y2Mn2O7 -> O2 + 4 YMnO3
0.5 Mn5O8 -> MnO + 1.5 MnO2
MnO + 1.5 MnO2 -> 0.5 Mn5O8
0.5 Mn3O4 -> MnO + 0.5 MnO2
MnO + 0.5 MnO2 -> 0.5 Mn3O4
2 MnO -> MnO2 + Mn
MnO2 + Mn -> 2 MnO
MnO2 + 0.5 Mn -> 0.5 Mn3O4
0.5 Mn3O4 -> MnO2 + 0.5 Mn


## 5. Conclusion

That's it! We hope this notebook was helpful in introducing the enumerator classes found in the _reaction-network_ package. If any significant errors are encountered, please first double-check that your settings are configured properly (e.g., proper installation of all dependencies, configuration of Materials Project API key, etc.). 

If the error persists, then please raise an Issue here: https://github.com/materialsproject/reaction-network/issues.

Happy enumerating!